In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-u5qj15vr
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-u5qj15vr
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=0b0e85b448ab3f443c8f6499355d05ab675c8b7e45b2378cde8906234702cc28
  Stored in directory: /tmp/pip-ephem-wheel-cache-x9iyuw4c/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [50]:
%%cuda --name LDF.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

#define THREADxBLOCK 128

using namespace std;

__global__ void colorer(Coloring * col, GraphStruct *str){
	int n = str->nodeSize;

	for (int i = threadIdx.x+blockIdx.x*blockDim.x; i < n; i += blockDim.x*gridDim.x) {
		bool flag = true; // vera sse il nodo ha peso locale massimo

		// ignora i nodi già colorati
		if ((col->coloring[i] != -1)) continue;

		int iWeight = str->weights[i];

		// guarda i pesi del vicinato
		uint offset = str->cumDegs[i];
		uint deg = str->cumDegs[i + 1] - str->cumDegs[i];

		for (uint j = 0; j < deg; j++) {
			uint neighID = str->neighs[offset + j];
			int jColor = col->coloring[neighID];

			if ((jColor != -1) || (i == neighID)) continue;

			int jWeight = str->weights[neighID];
			uint neighDeg = str->cumDegs[neighID + 1] - str->cumDegs[neighID];
			if ((deg < neighDeg) || ((deg == neighDeg) && (iWeight < jWeight))) flag = false;
		}

		// colora solo se sei il nodo di peso massimo
		if (flag){
			bool* forbidden;
			cudaMalloc((void**) &forbidden, n * sizeof(bool));
			memset(forbidden, false, n);

			for (uint j = 0; j < deg; j++) {
				uint neighID = str->neighs[offset + j];
				if(col->coloring[neighID] != -1) forbidden[col->coloring[neighID]] = true;
			}

			for(uint j = 0; j < n; j++){
				if(forbidden[j] == false){
					col->coloring[i] = j;
					break;
				}
			}
		free(forbidden);
		}
	}
}

void FYshuffle(int * weights, uint n){
    for(int i = 0; i < n; i++){
        int swapIdx = (rand() % (n - i)) + i;
        int tmp = weights[i];
				weights[i] = weights[swapIdx];
				weights[swapIdx] = tmp;
    }
}

Coloring* graphColoring(GraphStruct *str){
	int n = str->nodeSize;
	Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	CHECK(cudaMallocManaged(&(col->coloring), n * sizeof(int)));
	memset(col->coloring, -1 ,n * sizeof(int));

	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

	for (int i = 0; i < n; i++){
				str->weights[i] = i;
		}

	FYshuffle(str->weights, n);

	printf("Pesi: ");
	for(int i = 0; i < n; i++){
		printf("%d ", str->weights[i]);
	}
	printf("\n");

	//print_d <<< 1, 1 >>> (str, true);

	bool flag=true;
	while(flag){
		colorer<<<blocks, threads>>>(col, str);
		cudaDeviceSynchronize();
		flag=false;
		for(int i=0; i<n; i++){
			if(col->coloring[i]==-1){
				flag=true;
			}
		}
	}
  return col;
}

'File written in /content/src/LDF.cu'

In [51]:
%%cuda --name test_LDF.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"


int main(void) {
	unsigned int n = 1000;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

	cudaEventRecord(stop);
  cudaEventSynchronize(stop);

	//Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	uint maxColor = col->coloring[0];
 	printf("Coloratura trovata: ");
	for(int i = 0; i < str->nodeSize; i++){
			if(maxColor < col->coloring[i]) maxColor = col->coloring[i];
			printf("%d ", col->coloring[i]);
	}
	printf("\nColore massimo: %d", maxColor+1);

	return EXIT_SUCCESS;
}

'File written in /content/src/test_LDF.cu'

In [49]:
!nvcc -dc src/test_LDF.cu /content/src/LDF.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc test_LDF.o LDF.o graph.o graph_d.o -o test_LDF
!./test_LDF

Pesi: 641 930 509 42 799 795 938 931 529 237 725 658 630 581 334 29 895 801 453 547 247 380 126 840 399 594 90 605 568 847 460 312 320 768 624 893 341 812 532 227 912 300 229 731 385 304 928 152 587 388 504 609 146 272 764 168 769 894 543 62 883 235 720 735 750 99 195 206 178 626 844 661 357 25 971 202 503 239 468 553 785 561 653 240 259 289 72 323 904 340 867 462 990 492 269 197 954 650 276 903 490 75 541 295 798 249 58 108 113 636 218 611 145 423 410 677 607 814 550 939 993 564 631 772 997 92 998 951 977 863 913 305 299 644 684 325 866 697 853 932 454 342 396 458 905 417 856 800 787 117 50 762 350 303 147 16 730 336 432 861 9 875 638 918 548 781 436 705 231 766 198 376 393 974 172 355 437 646 319 767 217 28 420 76 803 713 444 254 531 574 647 753 67 602 896 643 888 63 952 662 989 61 967 555 308 74 979 424 363 821 934 220 603 483 562 274 683 560 709 257 26 434 876 484 714 656 124 412 597 189 645 972 387 390 473 394 148 391 520 228 114 882 537 837 379 233 858 186 623 749 268 942 324 361

In [47]:
!nvprof ./test_LDF

==21064== NVPROF is profiling process 21064, command: ./test_LDF
Pesi: 506 314 193 452 366 375 965 140 393 258 18 476 922 603 280 344 178 680 167 288 720 252 260 387 618 890 121 275 915 889 197 486 867 796 451 168 448 612 517 524 182 322 598 528 372 948 640 675 378 557 346 865 752 926 401 9 176 139 464 391 189 523 183 212 843 916 326 725 76 295 561 819 457 635 351 491 225 119 608 446 519 887 957 751 703 625 874 583 468 702 317 41 827 263 516 11 10 901 331 602 624 736 101 993 728 763 253 894 518 74 837 58 340 888 593 414 177 962 683 490 215 460 671 623 12 620 522 788 770 80 429 255 812 980 908 795 872 199 871 977 847 539 416 456 919 175 989 410 100 235 475 621 856 65 945 145 808 330 458 43 634 328 532 51 740 73 776 191 777 465 682 886 247 389 704 473 911 943 878 240 573 394 95 610 337 975 72 97 14 345 883 256 440 622 968 674 541 132 160 805 841 257 832 36 809 406 130 179 729 958 684 472 124 563 323 840 690 589 27 773 554 520 966 342 749 801 757 931 362 213 15 947 879 530 913 20 146 560 